In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [5]:
df = pd.read_csv("data/filter_twitter_data.csv")
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [6]:
X = df['stemmed_content'].astype(str)
X

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object

In [7]:
y = df['target']
y

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [9]:
vocab_size = 50000
max_len = 50

In [10]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [11]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [12]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [14]:
model_lstm = Sequential([
    Embedding(vocab_size, 128),
    LSTM(128, return_sequences=False),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [15]:
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
model_lstm.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
    ModelCheckpoint('best_lstm.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', patience=1, factor=0.5)
]

In [20]:
history_lstm = model_lstm.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=4,
    batch_size=512,
    callbacks=callbacks
)


Epoch 1/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.4998 - loss: 0.6933

2250/2250 ━━━━━━━━━━━━━━━━━━━━ 884s 393ms/step - accuracy: 0.4997 - loss: 0.6933 - val_accuracy: 0.4995 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 2/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 872s 387ms/step - accuracy: 0.4995 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 3/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.4994 - loss: 0.6932

2250/2250 ━━━━━━━━━━━━━━━━━━━━ 884s 393ms/step - accuracy: 0.4995 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931 - learning_rate: 5.0000e-04
Epoch 4/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step - accuracy: 0.5006 - loss: 0.6931

2250/2250 ━━━━━━━━━━━━━━━━━━━━ 1152s 512ms/step - accuracy: 0.5002 - loss: 0.6931 - val_accuracy: 0.5005 - val_loss: 0.6931 - learning_rate: 2.5000e-04


In [21]:
loss, acc = model_lstm.evaluate(X_test_pad, y_test)
print("LSTM Test Accuracy:", acc)


10000/10000 ━━━━━━━━━━━━━━━━━━━━ 158s 16ms/step - accuracy: 0.5000 - loss: 0.6932
LSTM Test Accuracy: 0.5


In [22]:
model_lstm.save('lstm_model.h5')

In [23]:
import pickle
with open('DL/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

## BiLSTM 

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

In [25]:
model_bilstm = Sequential([
    Embedding(vocab_size, 128),    
    Bidirectional(LSTM(128, return_sequences=False)),    
    Dropout(0.4),    
    Dense(64, activation='relu'),
    Dropout(0.3),    
    Dense(1, activation='sigmoid')
])

In [26]:
model_bilstm.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [27]:
model_bilstm.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
history_bilstm = model_bilstm.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=4,
    batch_size=512
)


Epoch 1/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 1717s 761ms/step - accuracy: 0.7725 - loss: 0.4740 - val_accuracy: 0.7850 - val_loss: 0.4513
Epoch 2/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 1673s 743ms/step - accuracy: 0.7979 - loss: 0.4323 - val_accuracy: 0.7880 - val_loss: 0.4485
Epoch 3/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 1670s 742ms/step - accuracy: 0.8117 - loss: 0.4063 - val_accuracy: 0.7878 - val_loss: 0.4594
Epoch 4/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 1667s 741ms/step - accuracy: 0.8247 - loss: 0.3804 - val_accuracy: 0.7850 - val_loss: 0.4826


In [29]:
loss, acc = model_bilstm.evaluate(X_test_pad, y_test)
print("BiLSTM Test Accuracy:", acc)


10000/10000 ━━━━━━━━━━━━━━━━━━━━ 203s 20ms/step - accuracy: 0.7858 - loss: 0.4790
BiLSTM Test Accuracy: 0.785812497138977


In [31]:
model_bilstm.save("bilstm_model.h5")

## GRU model

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

In [33]:
model_gru = Sequential([
    Embedding(vocab_size, 128),    
    GRU(128, return_sequences=False),    
    Dropout(0.4),    
    Dense(64, activation='relu'),
    Dropout(0.3),    
    Dense(1, activation='sigmoid')
])

In [34]:
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
model_gru.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
history_gru = model_gru.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=4,
    batch_size=512
)

Epoch 1/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 930s 412ms/step - accuracy: 0.4998 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 2/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 875s 389ms/step - accuracy: 0.4991 - loss: 0.6932 - val_accuracy: 0.5005 - val_loss: 0.6933
Epoch 3/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 876s 389ms/step - accuracy: 0.4998 - loss: 0.6932 - val_accuracy: 0.5005 - val_loss: 0.6931
Epoch 4/4
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 876s 389ms/step - accuracy: 0.4994 - loss: 0.6932 - val_accuracy: 0.5005 - val_loss: 0.6932


In [37]:
loss, acc = model_gru.evaluate(X_test_pad, y_test)
print("GRU Test Accuracy:", acc)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 135s 14ms/step - accuracy: 0.5000 - loss: 0.6932
GRU Test Accuracy: 0.5


In [39]:
model_gru.save("gru_model.h5")